In [1]:
# Change directory to workspace root so that relative path loads work correctly.
import os
try:
    os.chdir(os.path.join(os.getcwd(), '../'))
    print(os.getcwd())
except:
    pass
from ia.common.jira.connection import connect
import ia.common.jira.issue as ticket

from IPython.display import Markdown as md
import datetime
import json
import re

report_start_time = datetime.datetime.now()
report_timestamp = f'`Report executed at {report_start_time:%Y-%m-%d %H:%M}`'
md(report_timestamp)

/Users/lswolkien/Devel/EngineeringMetrics


`Report executed at 2020-06-16 09:49`

# Access variables

In [2]:
jira_url = os.environ['JIRA_URL']
jira_username = os.environ['JIRA_USERNAME']
jira_password = os.environ['JIRA_PASSWORD']


In [3]:
jira = connect(jira_url, basic_auth=(jira_username, jira_password))


In [4]:
issue_story = ticket.get_issue_by_key(jira, "DANSDP-1258")

In [5]:
for link in issue_story.fields.issuelinks:
    l_type = 'inwardIssue' if hasattr(link, 'inwardIssue') else 'outwardIssue'
    link.key = getattr(link, l_type).key
    print(f'{link.key} - {link.type.name} - {l_type}')

DANMR-144 - Blocks - outwardIssue
DANMR-520 - Blocks - outwardIssue
GPLAT-4145 - Blocks - inwardIssue
GPLAT-4105 - Blocks - inwardIssue
GPLAT-4148 - Blocks - inwardIssue
DANSDP-1400 - Related To - outwardIssue
DANSDP-1408 - Related To - inwardIssue
DANSDP-665 - Related To - inwardIssue


In [6]:
boards = [
    "Retention - Sprint board",
    "SDP Migration - Stories Board",
    "COE - Sprint board",
    "SDP Core - Sprint Board",
    "SDP Tools & Frameworks - Sprint Board",
    "MrGreen Core Board",
    "UNO Core - Sprint board"
]

In [49]:
board_sre = jira.boards(type="scrum", name="SRE - Sprint Board")

In [50]:
dir(board_sre)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'append',
 'clear',
 'copy',
 'count',
 'current',
 'extend',
 'index',
 'insert',
 'isLast',
 'iterable',
 'maxResults',
 'next',
 'pop',
 'remove',
 'reverse',
 'sort',
 'startAt',
 'total']

In [47]:
import re
for board_name in boards:
    board = jira.boards(type="scrum", name=board_name)[0]
    
    current_sprint = jira.sprints(board_id=board.id, state='active')[0]
    
    
    
    sprints = []
    startAt = 0
    while True:
        batch = jira.sprints(board_id=board.id, state='closed', startAt=startAt, maxResults=10)
        sprints += batch
        if len(batch) < 10:
            break
        startAt += len(batch)
    
    print(f'{board_name} ({len(sprints)})-> {sprints[0]} ({sprints[0].startDate[0:10]}) - {sprints[-1]} ({sprints[-1].startDate[0:10]}) ')
    
#     numbers_in_sprint_name = [int(s) for s in re.findall(r'\b\d+\b', current_sprint.name)][-1]
#     print(f'startAt = {numbers_in_sprint_name-len(sprints)}')
    for i, s in enumerate(sprints, start=1):
        print(f'{i}. {s}')
    print("==============")
    

Retention - Sprint board (19)-> DE Sprint 8 (2019-09-11) - Retention Sprint 26 (2020-05-20) 
1. DE Sprint 8
2. Retention Sprint 10
3. Retention Sprint 11
4. Retention Sprint 12
5. Retention Sprint 13
6. SDP-DI Sprint 13
7. Retention Sprint 14
8. Retention Sprint 15
9. Retention Sprint 16
10. Retention Sprint 17
11. Retention Sprint 18
12. Retention Sprint 19
13. Retention Sprint 20
14. Retention Sprint 21
15. Retention Sprint 22
16. Retention Sprint 23
17. Retention Sprint 24
18. Retention Sprint 25
19. Retention Sprint 26
SDP Migration - Stories Board (20)-> SDP- MrGreen Sprint 11 (2019-10-23) - SDP Core Platform Sprint 26 (2020-05-20) 
1. SDP- MrGreen Sprint 11
2. SDP - MrGreen  Sprint 12
3. SDP-DI2 Sprint 16
4. SDP-DI Sprint 17
5. SDP-DI Sprint 18
6. SDP-DI Sprint 19
7. SDP-DI Sprint 18
8. SDP-DI Sprint 20
9. SDP-DI Sprint 19
10. SDP - MrGreen Sprint 20
11. SDP-DI Sprint 21
12. SDP - MrGreen Sprint 21
13. SDP-DI Sprint 22
14. SDP Core Platform Sprint 24
15. SDP - MrGreen Sprint 22
1

In [48]:
id(sprints)

4420497232

In [33]:
print(sprints.total)

None


In [10]:
print(sprint.startDate)

2019-10-10T12:45:00.000Z


In [11]:
sprint.name


'UNO Sprint 10'

In [12]:
report_end_time = datetime.datetime.now()
report_timestamp = f'` Report finished at {report_end_time:%Y-%m-%d %H:%M}, time elapsed: {report_end_time - report_start_time}`'
md(report_timestamp)


` Report finished at 2020-06-16 09:49, time elapsed: 0:00:18.925869`